# Methods to comparing two DataFrames and find out differences

In [1]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

In [2]:
#Create data frame with sample data

Mon=[34,45,17,87]
Tue=[27,9,33,67]
Wed=[15,74,54,27]
Thu=[23,87,8,45]
Fri=[33,12,29,7]
index_col=['Steven','Mike','Andy','Paul']

sales=pd.DataFrame({'Mon':Mon,'Tue':Tue,'Wed':Wed,'Thu':Thu,'Fri':Fri},index=index_col)
sales

,Mon,Tue,Wed,Thu,Fri
Steven,34,27,15,23,33
Mike,45,9,74,87,12
Andy,17,33,54,8,29
Paul,87,67,27,45,7


In [3]:
#create a copy of another dataframe
sales1=sales.copy()

In [4]:
#update values in sales1
sales1.iloc[0,1]=100
sales1.iloc[3,2]=100

In [5]:
sales
sales1

,Mon,Tue,Wed,Thu,Fri
Steven,34,27,15,23,33
Mike,45,9,74,87,12
Andy,17,33,54,8,29
Paul,87,67,27,45,7


,Mon,Tue,Wed,Thu,Fri
Steven,34,100,15,23,33
Mike,45,9,74,87,12
Andy,17,33,54,8,29
Paul,87,67,100,45,7


In [6]:
#Solution1
sales==sales1

,Mon,Tue,Wed,Thu,Fri
Steven,True,False,True,True,True
Mike,True,True,True,True,True
Andy,True,True,True,True,True
Paul,True,True,False,True,True


In [7]:
#Solution 2
sales.where(~(sales==sales1))

,Mon,Tue,Wed,Thu,Fri
Steven,NaN,27.0,NaN,NaN,NaN
Mike,NaN,NaN,NaN,NaN,NaN
Andy,NaN,NaN,NaN,NaN,NaN
Paul,NaN,NaN,27.0,NaN,NaN


In [8]:
#Solution 3
sales1.where(~(sales==sales1))

,Mon,Tue,Wed,Thu,Fri
Steven,NaN,100.0,NaN,NaN,NaN
Mike,NaN,NaN,NaN,NaN,NaN
Andy,NaN,NaN,NaN,NaN,NaN
Paul,NaN,NaN,100.0,NaN,NaN


In [9]:
#Solution 4

sales_comp=pd.concat([sales,sales1],keys=['Day1','Day2'],axis=1)
sales_comp

Day1                 Day2                  
        Mon Tue Wed Thu Fri  Mon  Tue  Wed Thu Fri
Steven   34  27  15  23  33   34  100   15  23  33
Mike     45   9  74  87  12   45    9   74  87  12
Andy     17  33  54   8  29   17   33   54   8  29
Paul     87  67  27  45   7   87   67  100  45   7

In [10]:
def highlight_dff(data,color='yellow'):
    attr='background-color: {}'.format(color)
    other=data.xs('Day1',axis=1)
    return pd.DataFrame(np.where(data.ne(other,level=1),attr,''),index=data.index,columns=data.columns)

In [11]:
sales_comp.style.apply(highlight_dff,axis=None)

In [12]:
#Solution 5
def find_diff(df1,df2):
    assert(df1.columns==df2.columns).all()
    if any(df1.dtypes!=df2.dtypes):
        df2=df2.astypes(df1.dtypes)
    if df1.equals(df2):
        return None
    else:
        diff_data=((df1!=df2) & ~(df1.isnull() & df2.isnull()))
        stack=diff_data.stack()
        change_stack=stack[stack]
        change_stack.index.names=['id','col']
        diff_location=np.where(diff_data)
        changed_from=df1.values[diff_location]
        changed_to=df2.values[diff_location]
        return pd.DataFrame({'expected':changed_from,'actual':changed_to},index=change_stack.index)

In [13]:
find_diff(sales,sales1)

,,expected,actual
id,col,,
Steven,Tue,27,100
Paul,Wed,27,100
